# 0_ Initialisation de l'environnement colab.

In [ ]:
! pip install transformers

     |████████████████████████████████| 2.9 MB 13.3 MB/s 
     |████████████████████████████████| 636 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 61.9 MB/s 
     |████████████████████████████████| 3.3 MB 28.7 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from google.colab import drive
drive.mount('/drive')


Mounted at /drive


In [ ]:
drive_dir = '/drive/MyDrive'
data_dir = drive_dir + '/nlp_nature'

# 1_ Chargement des libraries

In [ ]:
import pandas as pd
import numpy as np
import torch

# 2_ Load data.

In [ ]:
conclusion_df=pd.read_csv(data_dir + '/jco_conclusion_df_bevacizumab_29092021.csv')
conclusion_df.head()

,Unnamed: 0,link,title,journal,authors,type,field,article,conclusion
0,0,https://ascopubs.org/doi/abs/10.1200/JCO.20.01225,\n\n \n \n\n ...,Journal of Clinical Oncology,"Chiara Cremolini,Search for articles by this a...",ORIGINAL REPORTS,Gastrointestinal Cancer,\n##Abstract##PURPOSE##A proper estimation of ...,FOLFOXIRI + bevacizumab significantly and mean...
1,1,https://ascopubs.org/doi/abs/10.1200/JOP.2015....,\n\n \n \n\n ...,Journal of Oncology Practice,"Stacie B. Dusetzina,Search for articles by thi...",Health Policy,Original Contributions,\n##Abstract##Section:##Choose##Top of page##A...,FOLFOXIRI + bevacizumab significantly and mean...
2,2,https://ascopubs.org/doi/abs/10.1200/JCO.2015....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Bradley J. Monk,Search for articles by this au...",ORIGINAL REPORTS,Gynecologic Cancer,\n##Abstract##Section:##Choose##Top of page##A...,"On the basis of the PFS, overall response rate..."
3,3,https://ascopubs.org/doi/abs/10.1200/JCO.19.03158,\n\n \n \n\n ...,Journal of Clinical Oncology,"Martin Reck,Search for articles by this author...",ORIGINAL REPORTS,Thoracic Oncology,\n##Abstract##Section:##Choose##Top of page##A...,ABCP seems tolerable and manageable versus ACP...
4,4,https://ascopubs.org/doi/abs/10.1200/JCO.2014....,\n\n \n \n\n ...,Journal of Clinical Oncology,"Miguel Martín,Search for articles by this auth...",ORIGINAL REPORTS,Breast Cancer,\n##Abstract##Section:##Choose##Top of page##A...,The addition of bevacizumab to ET in first-l...


# 3_ Load the transformers


We will use two models, provided by HuggingFace, both are making sentiment analysis.
siebert/sentiment-roberta-large-english is based on Roberta and is classifying text between positive/negative sentiment.

ProsusAI/finbert is based on Bert and is classifying text between positive/negative/neutral sentiment 

In [ ]:
from transformers import AutoTokenizer,  AutoModelForSequenceClassification


model_name_0 = "siebert/sentiment-roberta-large-english"
model_name_1 = "ProsusAI/finbert"

l_classifier_attributes = [{'name':'siebert', 'sentiments':['positive', 'negative']}, {'name':'finbert', 'sentiments':['positive', 'negative', 'neutral']}]
l_tokenizer = [AutoTokenizer.from_pretrained(model_name_0), AutoTokenizer.from_pretrained(model_name_1)]
l_classifiers = [AutoModelForSequenceClassification.from_pretrained(model_name_0), AutoModelForSequenceClassification.from_pretrained(model_name_1)]


Downloading:   0%|          | 0.00/256 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/687 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

In [ ]:
l_conclusion=conclusion_df['conclusion'].to_list()


# 4_ Traitement dun exemple de l_test

In [ ]:
# Transform input tokens
l_test = ['Stocks rallied and the British pound gained.', 'People seem to despise Apple', 'Performances of the firm are in the middle',
          'Stocks rallied and the British pound gained.', 'People seem to despise Apple', 'Performances of the firm are in the middle'] 
batch_size = 3
nb_batch = len(l_test) // batch_size 

In [ ]:
data_df = pd.DataFrame(data=[0,1,2,3,4,5], columns=['indice'])

In [ ]:
for i, attr in enumerate(l_classifier_attributes) :

  print(attr)
  name = attr['name']
  columns = attr['sentiments']
  for nb in range(nb_batch) :
     print(nb*batch_size, '*'*5, min([len(l_test),(nb+1)*batch_size]))
     l_run = l_test[nb*batch_size: min([len(l_test),(nb+1)*batch_size])]
     # Tokenize sentence list 
     input_tokens = l_tokenizer[i](l_run , truncation=True, padding=True, return_tensors="pt")

     # Model apply to get the missing logits
     outputs = l_classifiers[i](**input_tokens)


     #Transform the logits to probabilities
     softmax=torch.nn.Softmax(dim=1)  

     #Transform the logits tensor to numpy
     if nb == 0 :
       logits = outputs.logits.detach().numpy()
       probs=softmax(outputs.logits).detach().numpy()
     else :
       logits = np.concatenate((logits,outputs.logits.detach().numpy()), axis=0)
       probs = np.concatenate((probs,softmax(outputs.logits).detach().numpy()), axis=0)
     
     print(logits)
    
  #Add logits to the dataframe
  print(logits)
  data_df[[f'{name}_logit_{c}' for c in columns]] = logits
  #Add probs to the dataframe
  data_df[[f'{name}_prob_{c}' for c in columns]] = probs

{'name': 'siebert', 'sentiments': ['positive', 'negative']}
0 ***** 3
[[-3.7372289  2.9002604]
 [ 3.6077225 -3.0648067]
 [-2.7049448  1.9586798]]
3 ***** 6
[[-3.7372289  2.9002604]
 [ 3.6077225 -3.0648067]
 [-2.7049448  1.9586798]
 [-3.7372289  2.9002604]
 [ 3.6077225 -3.0648067]
 [-2.7049448  1.9586798]]
[[-3.7372289  2.9002604]
 [ 3.6077225 -3.0648067]
 [-2.7049448  1.9586798]
 [-3.7372289  2.9002604]
 [ 3.6077225 -3.0648067]
 [-2.7049448  1.9586798]]
{'name': 'finbert', 'sentiments': ['positive', 'negative', 'neutral']}
0 ***** 3
[[ 1.689791   -1.5705917  -0.90363085]
 [-1.6185646   1.8358628   0.9208008 ]
 [ 0.55224544 -1.3675247   0.75641626]]
3 ***** 6
[[ 1.689791   -1.5705917  -0.90363085]
 [-1.6185646   1.8358628   0.9208008 ]
 [ 0.55224544 -1.3675247   0.75641626]
 [ 1.689791   -1.5705917  -0.90363085]
 [-1.6185646   1.8358628   0.9208008 ]
 [ 0.55224544 -1.3675247   0.75641626]]
[[ 1.689791   -1.5705917  -0.90363085]
 [-1.6185646   1.8358628   0.9208008 ]
 [ 0.55224544 -1.367

In [ ]:
data_df

,indice,siebert_logit_positive,siebert_logit_negative,siebert_prob_positive,siebert_prob_negative,finbert_logit_positive,finbert_logit_negative,finbert_logit_neutral,finbert_prob_positive,finbert_prob_negative,finbert_prob_neutral
0,0,-3.737229,2.900260,0.001309,0.998691,1.689791,-1.570592,-0.903631,0.898362,0.034473,0.067165
1,1,3.607723,-3.064807,0.998736,0.001264,-1.618565,1.835863,0.920801,0.022069,0.698277,0.279654
2,2,-2.704945,1.958680,0.009344,0.990656,0.552245,-1.367525,0.756416,0.421381,0.061792,0.516827
3,3,-3.737229,2.900260,0.001309,0.998691,1.689791,-1.570592,-0.903631,0.898362,0.034473,0.067165
4,4,3.607723,-3.064807,0.998736,0.001264,-1.618565,1.835863,0.920801,0.022069,0.698277,0.279654
5,5,-2.704945,1.958680,0.009344,0.990656,0.552245,-1.367525,0.756416,0.421381,0.061792,0.516827


# 5_ Classification des conclusions

La RAM est trop faible pour traiter l'ensemble des conclusions simultanément, j'ai donc découpé en batch. Ca demeure un modèle gourmand en mémoire et en temps machine 265 s pour 453 sentences à traiter

In [ ]:
from tqdm import tqdm
import time

batch_size = 4
nb_batch = len(l_conclusion) // batch_size 

start = time.time()

for i, attr in enumerate(l_classifier_attributes) : 
  print(i)
  name = attr['name']
  columns = attr['sentiments']
  for nb in tqdm(range(nb_batch+1)) :

     l_run = l_conclusion[nb*batch_size: min([len(l_conclusion),(nb+1)*batch_size])]

     # Tokenize sentence list 
     input_tokens = l_tokenizer[i](l_run , truncation=True, padding=True, return_tensors="pt")

     # Model apply to get the missing logits
     outputs = l_classifiers[i](**input_tokens)

     #Transform the logits to probabilities
     softmax=torch.nn.Softmax(dim=1)  

     #Transform the logits tensor to numpy
     if nb == 0 :
       logits = outputs.logits.detach().numpy()
       probs=softmax(outputs.logits).detach().numpy()
     else :
       logits = np.concatenate((logits,outputs.logits.detach().numpy()), axis=0)
       probs = np.concatenate((probs,softmax(outputs.logits).detach().numpy()), axis=0)
  print(logits.shape)
  #Add logits to the dataframe
  conclusion_df[[f'{name}_logit_{c}' for c in columns]] = logits
  
  #Add probs to the dataframe
  conclusion_df[[f'{name}_prob_{c}' for c in columns]] = probs

end = time.time()

print(f'duree {end - start}')

0


100%|██████████| 114/114 [03:27<00:00,  1.82s/it]


(453, 2)
1


100%|██████████| 114/114 [00:57<00:00,  1.99it/s]

(453, 3)
duree 265.29846262931824


In [ ]:
conclusion_df.head()

<bound method NDFrame.head of      Unnamed: 0  ... finbert_prob_neutral
0             0  ...             0.032014
1             1  ...             0.032014
2             2  ...             0.250628
3             3  ...             0.627669
4             4  ...             0.040296
..          ...  ...                  ...
448         679  ...             0.806481
449         680  ...             0.806481
450         681  ...             0.578133
451         682  ...             0.145777
452         683  ...             0.145776

[453 rows x 19 columns]>

In [ ]:
conclusion_df.to_csv(data_dir + '/jco_scores_df_bevacizumab_30092021.csv')